# All Boosting Algorithms

Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold

Reading Our CSV Data

In [2]:
original = pd.read_csv("/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv")
train = pd.read_csv("/kaggle/input/playground-series-s3e3/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s3e3/test.csv")

# Analyzing Data

Mapping Data Correctly

In [3]:
original["Attrition"] = original["Attrition"].map({"Yes": 1, "No": 0})

In [4]:
train = train.drop(["id"], axis = 1)
test = test.drop(["id"], axis = 1)
original = original.drop(["EmployeeNumber"], axis = 1)

In [5]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   int64 
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EnvironmentSatisfaction   1470 non-null   int64 
 10  Gender                    1470 non-null   object
 11  HourlyRate                1470 non-null   int64 
 12  JobInvolvement            1470 non-null   int64 
 13  JobLevel                  1470 non-null   int64 
 14  JobRole                 

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1677 entries, 0 to 1676
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1677 non-null   int64 
 1   BusinessTravel            1677 non-null   object
 2   DailyRate                 1677 non-null   int64 
 3   Department                1677 non-null   object
 4   DistanceFromHome          1677 non-null   int64 
 5   Education                 1677 non-null   int64 
 6   EducationField            1677 non-null   object
 7   EmployeeCount             1677 non-null   int64 
 8   EnvironmentSatisfaction   1677 non-null   int64 
 9   Gender                    1677 non-null   object
 10  HourlyRate                1677 non-null   int64 
 11  JobInvolvement            1677 non-null   int64 
 12  JobLevel                  1677 non-null   int64 
 13  JobRole                   1677 non-null   object
 14  JobSatisfaction         

Both Original and Train csv match now, So we can Concatenate

In [7]:
train = pd.concat([train, original], axis = 0)

In [8]:
train.reset_index(inplace = True)
train = train.drop(["index"], axis = 1)

In [9]:
train.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
count,3147.000000,3147.000000,3147.000000,3147.000000,3147.0,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,...,3147.0,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000
mean,36.451223,850.586273,8.921513,2.926279,1.0,2.741023,66.907531,2.757865,2.029234,2.762313,...,80.0,0.755323,10.975850,2.763902,2.765809,6.900858,4.183985,2.076263,4.156975,0.138862
std,8.815861,390.859919,7.961278,1.032051,0.0,1.089733,19.877918,0.680183,1.094590,1.100224,...,0.0,0.810688,7.509438,1.214918,0.675707,5.998015,3.601691,3.130744,3.572003,0.345858
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,30.000000,1.000000,1.000000,1.000000,...,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,543.000000,2.000000,2.000000,1.0,2.000000,50.000000,2.000000,1.000000,2.000000,...,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000,0.000000
50%,35.000000,852.000000,7.000000,3.000000,1.0,3.000000,67.000000,3.000000,2.000000,3.000000,...,80.0,1.000000,9.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000,0.000000
75%,42.000000,1198.500000,13.000000,4.000000,1.0,4.000000,84.000000,3.000000,3.000000,4.000000,...,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,2.000000,7.000000,0.000000
max,60.000000,3921.000000,29.000000,15.000000,1.0,4.000000,100.000000,4.000000,7.000000,4.000000,...,80.0,3.000000,41.000000,6.000000,4.000000,41.000000,18.000000,15.000000,17.000000,1.000000


In [10]:
train.isnull().sum()

Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
Attrition 

Preprocessing Our Data

In [11]:
def preprocess(df):
    df = df.drop(["EmployeeCount", "Over18", "StandardHours"], axis=1)
    df["OverTime"] = df["OverTime"].map({"Yes": 1, "No": 0})
    df["Gender"] = df["Gender"].map({"Male": 1, "Female": 0})
    return df

In [12]:
train = preprocess(train)
test = preprocess(test)

In [13]:
train.tail()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
3142,36,Travel_Frequently,884,Research & Development,23,2,Medical,3,1,41,...,3,1,17,3,3,5,2,0,3,0
3143,39,Travel_Rarely,613,Research & Development,6,1,Medical,4,1,42,...,1,1,9,5,3,7,7,1,7,0
3144,27,Travel_Rarely,155,Research & Development,4,3,Life Sciences,2,1,87,...,2,1,6,0,3,6,2,0,3,0
3145,49,Travel_Frequently,1023,Sales,2,3,Medical,4,1,63,...,4,0,17,3,2,9,6,0,8,0
3146,34,Travel_Rarely,628,Research & Development,8,3,Medical,2,1,82,...,1,0,6,3,4,4,3,1,2,0


Extracting meaningful and correlated columns

In [14]:
corr_df = train.corr()

In [15]:
features = list(train.columns)
features.remove("Attrition")
target = "Attrition"

Encoding Our Object Data for both our Train and Test Data

In [16]:
def encode(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
    return df

In [17]:
train = encode(train)
test = encode(test)

For appending all our configuration models

In [18]:
clfs = []

----

# Extreme Gradient Boosting Model

Defining Our Train and Target Data

In [19]:
X = train.drop(["Attrition"], axis=1)
y = train["Attrition"]

Spliting Our Data into Train and Validation Sets

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Spliting Data into Train and Validation Sets

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Defining XGBoost Model

In [22]:
xgb = XGBRegressor()

Fitting and Predicting the Data

In [23]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [24]:
xgb_pred = xgb.predict(X_val)

Score for XGB Model

In [25]:
xgb_score = roc_auc_score(y_val, xgb_pred)

In [26]:
xgb_score

0.7664609053497943

XGB performs poorly than CBR and LGBR, so we won't consider it in the final clfs.

----

# CatBoostRegressor Model

Defining Train and Target Data

In [27]:
X = train.drop(["Attrition"], axis=1)
y = train["Attrition"]

Defining Stratified KFold for CBR

In [28]:
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

In [29]:
cbr_score = []

Spliting Data | Fitting the Model | Mean Score

In [30]:
for train_index, test_index in kfold.split(X, y):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    pool_train = Pool(X_train, y_train)
    pool_val = Pool(X_val, y_val)
    cbr = CatBoostRegressor(iterations = 200)
    cbr.fit(pool_train, eval_set =(X_val, y_val), verbose=False)
    cbr_preds = cbr.predict(X_val)    
    clfs.append(cbr)
    cbr_score.append(roc_auc_score(y_val, cbr_preds))
print(np.mean(cbr_score))

0.8464189869171419


----

# LightGradientBoostRegressor Model

Defining Train and Target Data

In [31]:
X = train.drop(["Attrition"], axis=1)
y = train["Attrition"]

Defining Stratified Kfold for LGBR

In [32]:
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

In [33]:
lgbr_score = []

Spliting Data | Fitting the Model | Mean Score

In [34]:
for train_index, test_index in kfold.split(X, y):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    lgbr = LGBMRegressor(n_estimators = 200)
    lgbr.fit(X_train, y_train)
    lgbr_preds = lgbr.predict(X_val)
    clfs.append(lgbr)
    lgbr_score.append(roc_auc_score(y_val, lgbr_preds))
print(np.mean(lgbr_score))

0.8125092640989834


----

# Predicting Validation Results 

Defining Train and Target Data

In [35]:
X = train.drop(["Attrition"], axis=1)
y = train["Attrition"]

Predict Score

In [36]:
for clf in clfs:
    pred = clf.predict(X)
    score = roc_auc_score(y, pred)

In [37]:
score

0.9841024428550922

# Predicting Test Results on Ensembled Models

In [38]:
for clf in clfs:
    clf_pred = clf.predict(test)

# Submissions

Reading Submissions.csv

In [39]:
submission = pd.read_csv("/kaggle/input/playground-series-s3e3/sample_submission.csv", usecols = ["id"])

In [40]:
submission["Attrition"] = clf_pred

Saving Our Final submission.csv

In [41]:
submission.to_csv("submission.csv", index = False)

----